# Much of the exam was built outside this notebook

In [1]:
import pandas as pd
import numpy as np

import yfinance as yf

/Users/markhendricks/Projects/finm-portfolio-2023/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
TICKS = [
    'USO',
    '^TNX',
]

STARTDATE = '2009-03-31'
ENDDATE = '2023-11-30'

df = yf.download(TICKS, start=STARTDATE, end=ENDDATE)['Adj Close']

prices = df.resample('M').last()

[*********************100%%**********************]  2 of 2 completed


In [3]:
prices

,USO,^TNX
Date,,
2009-03-31,232.399994,2.685
2009-04-30,229.039993,3.124
2009-05-31,291.200012,3.465
2009-06-30,303.440002,3.523
2009-07-31,294.480011,3.501
...,...,...
2023-07-31,73.169998,3.959
2023-08-31,75.059998,4.093
2023-09-30,80.860001,4.573


In [4]:
fore = prices.pct_change().dropna()

fore['Tnote rate'] = prices['^TNX']
fore.drop(columns=['^TNX'],inplace=True)
fore['Tnote rate change'] = fore['Tnote rate'].diff()
fore.dropna(inplace=True)

***

In [5]:
import pandas_datareader.data as web
import pandas_datareader as pdr

In [6]:
START_DATE = '2018-04-01'
END_DATE = '2023-11-30'
TICKS_FX = ['DEXMXUS']#'DEXUSEU', 'DEXUSUK',
TICKS_RF = ['SOFR']#'ECBESTRVOLWGTTRMDMNRT','IUDSOIA', 'IR3TIB01MXM156N',
LABS_RF = ['SOFR']#'ESTR', 'SONIA'

INDIRECT = False

fxraw = pdr.DataReader(TICKS_FX,data_source='fred', start=START_DATE,end=END_DATE)
fxraw['DEXMXUS'] = 1 / fxraw['DEXMXUS']

map_labels = {TICKS_RF[i]:LABS_RF[i] for i in range(len(TICKS_RF))}
rf = pdr.DataReader(TICKS_RF,data_source='fred', start=START_DATE,end=END_DATE).rename(columns=map_labels).dropna()/100

In [7]:
rfmxn = pd.read_excel('../data/fx_mxn_rf.xlsx',index_col='Date').sort_index().rename(columns={'PX_LAST':'MXNRF'})/100
rfall = pd.concat([rf,rfmxn],join='inner',axis=1)/252

In [8]:
fxrfdata = pd.concat([fxraw,rfall],join='inner',axis=1)

***

In [9]:
retsx = pd.read_excel('../data/spx_returns_weekly.xlsx',sheet_name='benchmark rets',index_col='date')

***

# Write File

In [10]:
with pd.ExcelWriter('../data/final_exam_data_raw.xlsx') as writer:  
    retsx.to_excel(writer, sheet_name='portfolio')
    fore.to_excel(writer, sheet_name='forecasting')
    fxrfdata.to_excel(writer,sheet_name='fx_carry')